# Lab 1
exercises in python using minizinc library

---

**How to use special html syntax**
<ul>
    <li><input type="checkbox" checked>Checked element demo.</li>
    <li><input type="checkbox">Unchecked element demo.</li>
</ul>

Highlighted inline formula demo: <span style="background-color: yellow;">*$[X_1, X_2, \ldots, X_n]$*</span>

---

## The N-queens problem
Place n queens in an nxn board so that no 
two queens can attack each other. 
<ul>
    <li><input type="checkbox"> Implement the 3 models with GAC on alldifferent.</li>
    <li><input type="checkbox"> Search for all solutions for n = 8, 9, 10, 12 using the default search of Gecode.</li>
    <li><input type="checkbox"> Compare the number of solutions and the failures.</li>
</ul>



### 1. Row Model

**Variables and Domains:**

- A variable for each row $[X_1, X_2, ..., X_n]$ → no row attack  
- Domain values $[1..n]$ represent the columns:
  - $X_i = j$ means that the queen in row $i$ is in column $j$

**Constraints:**

- `alldifferent([X_1, X_2, ..., X_n])` → no column attack  
- For all $i < j$, $|X_i - X_j| \ne |i - j|$ → no diagonal attack

In [9]:
%load_ext iminizinc

The iminizinc extension is already loaded. To reload it, use:
  %reload_ext iminizinc


In [10]:
%%writefile "./n-Queens files/queens_row_model.mzn"
include "alldifferent.mzn";

int: n;
array[1..n] of var 1..n: rows; % variables & domains 

% column attack constraints
constraint alldifferent(rows);

% diagonal attack constraints
constraint forall(i, j in 1..n where i < j)(
    abs(rows[i] - rows[j]) != abs(i - j)
);

solve satisfy;

Overwriting ./n-Queens files/queens_row_model.mzn


In [11]:
for n in [8, 9, 10, 12]:
    print(f"\nResult for n = {n}")
    !minizinc "./n-Queens files/queens_row_model.mzn" -D n={n}



Result for n = 8
rows = [4, 2, 7, 3, 6, 8, 5, 1];
----------

Result for n = 9
rows = [6, 4, 9, 5, 8, 2, 7, 3, 1];
----------

Result for n = 10
rows = [6, 8, 5, 2, 4, 10, 7, 9, 3, 1];
----------

Result for n = 12
rows = [11, 2, 5, 7, 4, 10, 12, 9, 6, 8, 3, 1];
----------


### 2. AllDifferent Model 
**Variables**
- $[X_1, X_2, ..., X_n] \in [1..n]$

**Constraints**
- alldifferent($[X_1, X_2, ..., X_n]$)
- alldifferent($[X_1 + 1,  X_2 + 2, ..., X_n + n]$)
- alldifferent($[X_1 – 1, X_2 – 2, ..., X_n – n]$)


In [18]:
%%writefile "./n-Queens files/queens_alldiff_model.mzn"
include "alldifferent.mzn";

int: n;
array[1..n] of var 1..n: rows; % variables & domains 

% column attack constraints
constraint alldifferent(rows);

% diagonal attack constraints
constraint alldifferent([rows[i] + i | i in 1..n]);
constraint alldifferent([rows[i] - i | i in 1..n]);

solve satisfy;

Overwriting ./n-Queens files/queens_alldiff_model.mzn


In [21]:
for n in [8, 9, 10, 12]:
    print(f"\nResult for n = {n}")
    !minizinc "./n-Queens files/queens_alldiff_model.mzn" -D n={n} --statistics


Result for n = 8
% Generated FlatZinc statistics:
%%%mzn-stat: paths=0
%%%mzn-stat: flatIntVars=24
%%%mzn-stat: flatIntConstraints=19
%%%mzn-stat: method="satisfy"
%%%mzn-stat: flatTime=0.13853
%%%mzn-stat-end
rows = [4, 2, 7, 3, 6, 8, 5, 1];
----------
%%%mzn-stat: initTime=0.0010942
%%%mzn-stat: solveTime=0.000345
%%%mzn-stat: solutions=1
%%%mzn-stat: variables=24
%%%mzn-stat: propagators=19
%%%mzn-stat: propagations=1374
%%%mzn-stat: nodes=47
%%%mzn-stat: failures=22
%%%mzn-stat: restarts=0
%%%mzn-stat: peakDepth=5
%%%mzn-stat-end
%%%mzn-stat: nSolutions=1
%%%mzn-stat-end

Result for n = 9
% Generated FlatZinc statistics:
%%%mzn-stat: paths=0
%%%mzn-stat: flatIntVars=27
%%%mzn-stat: flatIntConstraints=21
%%%mzn-stat: method="satisfy"
%%%mzn-stat: flatTime=0.14925
%%%mzn-stat-end
rows = [5, 7, 9, 4, 2, 8, 6, 3, 1];
----------
%%%mzn-stat: initTime=0.0006765
%%%mzn-stat: solveTime=0.0001882
%%%mzn-stat: solutions=1
%%%mzn-stat: variables=27
%%%mzn-stat: propagators=21
%%%mzn-stat: pr

### 3. Alldifferent and Symmetry Breaking Model
**Variables and Domains:**
- for all $i, X_i \in [1..n]$, for all $i, j B_{ij} \in [0..1]$

**Constraints:**
- alldifferent($[X_1, X_2, ..., X_n]$) 
- alldifferent($[X_1 + 1,  X_2 + 2, ..., X_n + n]$)
- alldifferent($[X_1 – 1, X_2 – 2, ..., X_n – n]$) 
- lex≤(B , π(B)) for all π
- _Channeling Constraints_

    - for all $i,j X_i = j B_{ij} = 1 $

[ Section 2.7.6 of the MiniZinc Tutorial ]

In [16]:
%%writefile "./n-Queens files/queens_alldiff_sym_model.mzn"
include "alldifferent.mzn";

int: n;
array[1..n] of var 1..n: X; % variables & domains 

% Channeling matrix: B[i,j] = 1 iff X[i] = j
array[1..n, 1..n] of var bool: B;

% column attack constraints
constraint alldifferent(X);

% diagonal attack constraints
constraint alldifferent([X[i] + i | i in 1..n]);
constraint alldifferent([X[i] - i | i in 1..n]);

% Channeling constraints
constraint forall(i in 1..n, j in 1..n)(
    (X[i] = j) <-> (B[i,j])
);

% Symmetry breaking: lex-leader (fix first queen in first half columns)
constraint X[1] <= ceil(n/2);

solve satisfy;

Writing ./n-Queens files/queens_alldiff_sym_model.mzn


In [ ]:
#TODO check if the last minizinc model is correct !! 
#TODO print also some statistics when executing the code

for n in [8, 9, 10, 12]:
    print(f"\nResult for n = {n}")
    !minizinc "./n-Queens files/queens_alldiff_sym_model.mzn" -D n={n}


Result for n = 8
X = [1, 7, 4, 6, 8, 2, 5, 3];
B = 
[|  true, false, false, false, false, false, false, false
 | false, false, false, false, false, false,  true, false
 | false, false, false,  true, false, false, false, false
 | false, false, false, false, false,  true, false, false
 | false, false, false, false, false, false, false,  true
 | false,  true, false, false, false, false, false, false
 | false, false, false, false,  true, false, false, false
 | false, false,  true, false, false, false, false, false
 |];
----------

Result for n = 9
X = [1, 4, 7, 9, 2, 5, 8, 6, 3];
B = 
[|  true, false, false, false, false, false, false, false, false
 | false, false, false,  true, false, false, false, false, false
 | false, false, false, false, false, false,  true, false, false
 | false, false, false, false, false, false, false, false,  true
 | false,  true, false, false, false, false, false, false, false
 | false, false, false, false,  true, false, false, false, false
 | false, false, fals

In [ ]:
#TODO idea to capture time, number of solutions and number of fails in order to compare those datas and get to a conclusion
import subprocess
import re
import time

# The values of n you want to test
values = [8, 9, 10, 12]

for n in values:
    print(f"\n=== n = {n} ===")

    # Measure wall time
    start = time.time()
    
    # Run MiniZinc with all solutions and statistics
    result = subprocess.run(
        [
            "minizinc",
            "./n-Queens files/queens_row_model.mzn",
            "-D", f"n={n}",
            "--all-solutions",
            "--statistics"
        ],
        capture_output=True, text=True
    )
    end = time.time()

    output = result.stdout

    # ---- Parse output ----
    # Count number of solutions (each ends with "----------")
    num_solutions = output.count("----------")
    
    # Extract number of failures (if present)
    match_failures = re.search(r"%%%mzn-stat: failures=(\d+)", output)
    failures = int(match_failures.group(1)) if match_failures else None

    # Extract solver time (MiniZinc’s internal time)
    match_time = re.search(r"%%%mzn-stat: time=([0-9.]+)", output)
    solver_time = float(match_time.group(1)) if match_time else None

    # Print summary
    print(f"Solutions: {num_solutions}")
    print(f"Failures:  {failures}")
    print(f"Solver time: {solver_time:.3f}s" if solver_time else "Solver time: N/A")
    print(f"Wall time (Python): {end - start:.3f}s")
